# Caso di studio

In [1]:
import pandas as pd

## Caso di studio

Spesso è necessario fondere più DataFrame per estrarre informazioni dai dati. Vedremo adesso alcuni casi esemplificativi sui dati relativi al sistema universitario.

Calcoliamo, per ogni Ateneo, il rapporto fra il totale delle spese e il numero totali di studenti iscritti. Solo alcune righe di `iscritti` riportano il numero totale di studenti iscritte, le altre righe non sono interessanti per questa analisi. Controlliamo i valori presenti nella colonna `DESCRIZIONE_ISCRIZIONE` per capire come selezionare le righe che interessano.

In [30]:
iscritti['DESCRIZIONE_ISCRIZIONE'].unique()

array(['Totale iscritti',
       'Totale iscritti - di cui assegnatari di borsa di studio di dottorato di ricerca (DM 30/4/99 n. 224, art 7)',
       'Totale iscritti - di cui studenti con parziale copertura delle tasse da enti esterni',
       'Totale iscritti - di cui studenti iscritti a tempo parziale'],
      dtype=object)

Quindi dobbiamo selezionare le righe dove `DESCRIZIONE_ISCRIZIONE` è uguale a `Totale iscritti`.

## Iscritti di ogni Ateneo

Selezioniamo gli studenti iscritti di ogni Ateneo

In [41]:
iscritti_ateneo = iscritti[iscritti['DESCRIZIONE_ISCRIZIONE'] == 'Totale iscritti']
iscritti_ateneo.head(4)

,ANNO_ACCADEMICO,COD_ATENEO,NOME_ATENEO,CODICE_ISCRIZIONE,DESCRIZIONE_ISCRIZIONE,ISCRITTI_LAUREA,ISCRITTI_DOTTORATO,ISCRITTI_SPECIALIZZAZIONE,ISCRITTI_MASTER_PERFEZIONAMENTO
0,2008-2009,101,Torino - Università degli studi,T1,Totale iscritti,63494.0,1255.0,1982.0,1227.0
1,2008-2009,102,Torino - Politecnico,T1,Totale iscritti,25399.0,689.0,30.0,260.0
2,2008-2009,201,Vercelli - Università degli studi del Piemonte...,T1,Totale iscritti,9470.0,182.0,267.0,175.0
3,2008-2009,401,Bra (CN) - Università di Scienze Gastronomiche,T1,Totale iscritti,229.0,0.0,0.0,24.0


## Iscritti di ogni Ateneo 2

Una veloce  analisi del nuovo DataFrame (ad esempio guardando l'insieme delle colonne) mostra che esistono 4 tipologie di iscritti e che ogni Ateneo è presente in più righe, una per ogni anno.

Per evitare di avere un'analisi influenzata eccessivamente questo ultimo aspetto (ad esempio, l'università *Roma - Link Campus University* compare in una sola riga):
1.  costruiamo una nuova colonna `iscritti` con il totale degli iscritti;
2.  calcoliamo il numero medio di iscritti per ogni ateneo.

In [45]:
iscritti_ateneo.loc[:, 'iscritti'] = iscritti_ateneo['ISCRITTI_LAUREA'] + \
                              iscritti_ateneo['ISCRITTI_DOTTORATO'] + \
                              iscritti_ateneo['ISCRITTI_SPECIALIZZAZIONE'] + \
                              iscritti_ateneo['ISCRITTI_MASTER_PERFEZIONAMENTO']

## Numero medio di iscritti

Adesso possiamo calcolare il numero medio di iscritti per ogni Ateneo

In [69]:
iscritti_medio = iscritti_ateneo.groupby('COD_ATENEO', 
                                         as_index = False).mean()[['COD_ATENEO', 'iscritti']]
iscritti_medio.head(2)

,COD_ATENEO,iscritti
0,101,68216.0
1,102,28546.8


Passiamo ad analizzare il DataFrame `gettito`, a partire dalle sue colonne.

In [49]:
gettito.columns

Index(['ANNO_SOLARE', 'COD_Ateneo', 'NOME_ATENEO', 'CODICE_GETTITO',
       'DESCRIZIONE_GETTITO', 'CONSUNTIVO'],
      dtype='object')

## Gettito medio

Per coerenza con quanto fatto per gli iscritti, andiamo a calcolare per ogni Ateneo, il gettito medio (calcolando la media sugli anni solari), in modo da rendere omogenei i dati su Atenei diversi.

Ciò richiede due passaggio:
1.  per ogni Ateneo e ogni anno, calcolare il gettito totale
2.  per ogni Ateneo, calcolare il gettito medio. Questo punto richiede che `COD_Ateneo` sia una variabile, non parte dell'indice, del DataFrame ottenuto al punto precedente.

In [50]:
gettito_totale = gettito.groupby(['COD_Ateneo', 'ANNO_SOLARE'], as_index = False).sum()[['COD_Ateneo', 'ANNO_SOLARE', 'CONSUNTIVO']]

In [51]:
gettito_medio = gettito_totale.groupby('COD_Ateneo', as_index = False).mean()[['COD_Ateneo', 'CONSUNTIVO']]

In [63]:
gettito_medio.head(2)

,COD_Ateneo,CONSUNTIVO
0,101,8.825587e+07
1,102,2.862463e+07


## Calcolo rapporto

Per calcolare il rapporto desiderato, dobbiamo fondere i DataFrame. Sul risultato possiamo poi aggiungere la nuova colonna.

In [70]:
fuso = pd.merge(gettito_medio, iscritti_medio, left_on = 'COD_Ateneo', right_on = 'COD_ATENEO')
fuso.head()

,COD_Ateneo,CONSUNTIVO,COD_ATENEO,iscritti
0,101,8.825587e+07,101,68216.0
1,102,2.862463e+07,102,28546.8
2,201,9.896148e+06,201,10082.4
3,401,3.921876e+06,401,301.2
4,701,9.898892e+05,701,1148.6


## Calcolo rapporto 2

Adesso calcoliamo la colonna `rapporto`.

In [71]:
fuso['rapporto'] = fuso['CONSUNTIVO'] / fuso['iscritti']
fuso.head(4)

,COD_Ateneo,CONSUNTIVO,COD_ATENEO,iscritti,rapporto
0,101,8.825587e+07,101,68216.0,1293.770769
1,102,2.862463e+07,102,28546.8,1002.726255
2,201,9.896148e+06,201,10082.4,981.527057
3,401,3.921876e+06,401,301.2,13020.837981


## Verifica fusione

E' sempre opportuno verificare la correttezza di una `merge`: in questo caso quella che ha prodotto il DataFrame `gettito_fuso`.

In particolare, bisogna:
1.  eseguire la merge con `how` uguale a `outer` per verificare che non ci siano atenei non presenti in `gettito_fuso`
2.  aggiungere l'opzione `validate` per verificare che non ci siano atenei presenti in più righe.
3.  controllare che il risultato di questa nuova `merge` abbia lo stesso numero di righe di `gettito_fuso`

In [72]:
completo = pd.merge(gettito_medio, iscritti_medio, left_on = 'COD_Ateneo', right_on = 'COD_ATENEO',
                    validate = '1:1',
                    how = 'outer')
len(fuso) == len(completo)

True

## Completamento risultati

Il DataFrame `gettito_fuso` ha il rapporto desiderato ma, nei vari passaggi, abbiamo perso traccia dei nomi degli Atenei (abbiamo solo il codice). Diventa necessaria una nuova fusione per reperire questi dati.

Siccome non abbiamo un DataFrame con i soli nomi degli Atenei, dobbiamo crearlo a partire (ad esempio) dal DataFrame `iscritti`.

In [56]:
atenei = iscritti[['COD_ATENEO', 'NOME_ATENEO']].drop_duplicates()
atenei.head()

,COD_ATENEO,NOME_ATENEO
0,101,Torino - Università degli studi
1,102,Torino - Politecnico
2,201,Vercelli - Università degli studi del Piemonte...
3,401,Bra (CN) - Università di Scienze Gastronomiche
4,701,Aosta - Università degli studi


## Completamento risultati 2

Controlliamo se esistono due righe riferite allo stesso codice di Ateneo.

In [57]:
atenei.groupby('COD_ATENEO').count().max()

NOME_ATENEO    3
dtype: int64

Siccome esiste almeno un Ateneo memorizzato con due nomi diversi, dobbiamo ulteriormente modificare `atenei` per associare un singolo nome ad ogni codice.

## Completamento risultati 3

Per associare un singolo nome ad ogni codice possiamo usare la funzione `first` che estrae la prima riga di ogni gruppo.

In [58]:
atenei_puliti = atenei.groupby('COD_ATENEO').first()
atenei_puliti.head()

,NOME_ATENEO
COD_ATENEO,
101,Torino - Università degli studi
102,Torino - Politecnico
201,Vercelli - Università degli studi del Piemonte...
401,Bra (CN) - Università di Scienze Gastronomiche
701,Aosta - Università degli studi


## Completamento risultati 4

Per associare un singolo nome ad ogni codice possiamo usare la funzione `first` che estrae la prima riga di ogni gruppo.

Finalmente possiamo fondere `atenei_puliti` con `gettito_fuso` e tenere solo le colonne interessanti.

In [59]:
risultato = pd.merge(atenei_puliti, fuso, left_index = True, right_on = 'COD_ATENEO')
del(risultato['COD_Ateneo'])
del(risultato['COD_ATENEO'])
risultato.head()

,NOME_ATENEO,CONSUNTIVO,iscritti,rapporto
0,Torino - Università degli studi,8.825587e+07,68216.0,1293.770769
1,Torino - Politecnico,2.862463e+07,28546.8,1002.726255
2,Vercelli - Università degli studi del Piemonte...,9.896148e+06,10082.4,981.527057
3,Bra (CN) - Università di Scienze Gastronomiche,3.921876e+06,301.2,13020.837981
4,Aosta - Università degli studi,9.898892e+05,1148.6,861.822392


## Completamento risultati 5

Per completare lo studio, andiamo ad identificare gli atenei che hanno rapporto massimo e minimo, oltre a calcolare le statistiche descrittive su `risultato`.

In [60]:
risultato.loc[risultato['rapporto'].idxmax()]

NOME_ATENEO    Bra (CN) - Università di Scienze Gastronomiche
CONSUNTIVO                                        3.92188e+06
iscritti                                                301.2
rapporto                                              13020.8
Name: 3, dtype: object

In [61]:
risultato.loc[risultato['rapporto'].idxmin()]

NOME_ATENEO    Lucca - Scuola IMT Alti Studi
CONSUNTIVO                                 0
iscritti                               109.2
rapporto                                   0
Name: 38, dtype: object

## Calcolo statistiche di basi

Adesso possiamo iniziare l'esplorazione dei dati, calcolando alcune statistiche basilari.

In [62]:
risultato.describe()

,CONSUNTIVO,iscritti,rapporto
count,9.700000e+01,97.000000,97.000000
mean,2.513951e+07,19858.642268,1679.972634
std,3.320958e+07,23424.355577,1977.583727
min,0.000000e+00,109.200000,0.000000
25%,5.618214e+06,3138.400000,736.912961
50%,1.603061e+07,11104.000000,1090.934317
75%,3.077572e+07,29635.600000,1623.641593
max,1.809762e+08,136846.600000,13020.837981
